<a href="https://colab.research.google.com/github/nprimavera/Digital-Manufacturing/blob/main/Fractal_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Vectors

Notes:
- vectors can have various numbers of entries
- in this lab we will only deal with vectors that have two entries
- such vectors can be used to represent points on a plane, or pixels in the GP142 graphics window
- we simply take the first entry in each vector to be the x-coordinate of a point, and the second entry to be the y-coordinate
- adding two vectors a and b results in a new vector c whose first entry is the sum of the first entries of a and b and whose second entry is the sum of the second entries of a and b --> (1.0, 2.0) + (3.0, 4.0) = (1.0+3.0, 2.0+4.0) = (4.0, 6.0)



In [ ]:
# Matrices

Notes:
- a matrix is a two-dimensional grid of numbers laid out in rows and columns
- in this lab we will only deal with 2-by-2 matrices
- 2-by-2 matrices have two rows and two columns and thus contain four entries
- in this lab we will refer to these four entries as a, b, c, and d --> these entries are laid out from left to right and top to bottom
- a 2-by-2 matrix and a 2-entry vector can be multiplied --> result is a new vector --> ex. A(matrix) * v(vector) = (a*x+b*y, c*x+d*y)

In [ ]:
# Linear Transformation

Notes:
- if the vector represents a point on the screen, then multiplying the vector by a matrix generates a new point on the screen
- the nature of the transformation depends on the specific entries of the matrix --> linear transformation
- given a whole set of vector points, applying a linear transformation can rotate, scale, shear, and reflect the points in the set --> ex: if the points represent an image on screen, a linear transformation might enlarge, stretch, skew, or mirror the image

In [ ]:
# Affine Transformation

Notes:
- a linear transformation cannot be used to translate a set of points -- > affine transformation
- affine transformation multiplies a vector by a matrix, just as in a linear transformation, and then adds a vector to the result --> this added vector carries out the translation
- by applying an affine transformation to an image on the screen we can do everything a linear transformation can do, and also have the ability to move the image up or down and left or right


Process:
- declare a struct AffineTransformation that contains a matrix and a vector
- then define a function affineTransformation that takes an AffineTransformation pointer as its first parameter, and a Vector pointer as its second paramter
- this function should apply the AffineTransformation to the Vector and return a new Vector that is the result of the transformation
- it should do this by making calls to multiply and add

In [ ]:
def create_svg(affineTransformation):
    svg_content = '''

    // Assuming you have a Vector struct defined somewhere
    typedef struct {    // contains three double-precision floating-point variables (x,y,z) representing the coordinates of the vector
        double x;
        double y;
        double z;
    } Vector;           // a Vector struct is defined, which represents a point or a vector in 3D space

    // Define your AffineTransformation struct
    typedef struct {
        double matrix[3][3];    // 3x3 matrix - represents the linear transformation
        Vector vector;          // vector - represents the translation
    } AffineTransformation;

    // Function to multiply a matrix with a vector
    Vector multiply(const double matrix[3][3], const Vector *vector) {
        Vector result;
        // visualize: matrix row * vector column
        result.x = matrix[0][0] * vector->x + matrix[0][1] * vector->y + matrix[0][2] * vector->z;
        result.y = matrix[1][0] * vector->x + matrix[1][1] * vector->y + matrix[1][2] * vector->z;
        result.z = matrix[2][0] * vector->x + matrix[2][1] * vector->y + matrix[2][2] * vector->z;
        return result;
    }

    // Function to add two vectors
    Vector add(const Vector *v1, const Vector *v2) {
        Vector result;
        result.x = v1->x + v2->x;
        result.y = v1->y + v2->y;
        result.z = v1->z + v2->z;
        return result;    // this function takes two vectors and returns their sum as a new vector
    }

    // Function to apply the AffineTransformation to the Vector
    // the function 'affineTransformation' takes an 'AffineTransformation' pointer 'affine' and a 'Vector' pointer 'vector'
    Vector affineTransformation(const AffineTransformation *affine, const Vector *vector) {   // apply the affine transformation represented by 'affine' to the vector 'vector'
        Vector transformed_vector = multiply(affine->matrix, vector);   // multiply the matrix part of the affine transformation with the vector
        return add(&transformed_vector, &affine->vector);               // add the translation part --> returns the resulting vector
    }

    int main() {
        // Example usage
        AffineTransformation transformation = {           // creates an 'AffineTransformation' object called 'transformation'
            .matrix = {{1, 0, 0}, {0, 1, 0}, {0, 0, 1}},  // identity matrix as the transformation matrix
            .vector = {1, 2, 3}                           // translation vector {1, 2, 3}
        };
        Vector input_vector = {1, 2, 3};                  // input vector {1, 2, 3}
        Vector result_vector = affineTransformation(&transformation, &input_vector);  // applies the affine transformation to the input vector using the 'affineTransformation function
        printf("Transformed vector: (%.2f, %.2f, %.2f)\n", result_vector.x, result_vector.y, result_vector.z);  // prints the resulting transformed vector
        return 0;
    }
    '''

    # Write the SVG content to the file
    with open(affineTransformation, 'w') as svg_file:  # Opens a file for writing. Creates the file if it does not exist. Overwrites the file if it exists.
        svg_file.write(svg_content)

    # Print the SVG content to the console
    #print(svg_content)

# Example usage
create_svg("affineTransformation.svg")   # creates an SVG file called example.svg

In [ ]:
# Fractals

Notes:
- a fractal is an object that displays self-similarity at magnified scales
- there are many objects in nature that have fractal properties --> ex: the branching structure of the tiny capillaries of the human circulatory system is similar in structure to the branching of the large arteries that feed them; the same is true of the nervous system, tree limbs and root systems, and river systems, and another example of fractals in nature is the similarity in appearance of a rugged mountain from a distance and a tiny pebble on that mountain viewed from close-up
- Fractal objects are found everywhere in mathematics as well

Process:
- in this lab we will use vector transformations to generate them
- to draw a fractal image we will simply start with a single vector and apply a set of affine transformations to it to generate a new set of vectors
- we will then recursively apply the same set of transformations to these new vectors, and so on, until we have many thousands of vectors
- then we will plot these points on the screen to generate an image --> use the power of recursive functions to make it simple

In [ ]:
# example from https://courses.cs.washington.edu/courses/cse142/01sp/misc/fractal_lab.htm
def create_svg(fractal):
    svg_content = '''
    int main(void)
    {
        int mouse_x, mouse_y;
        char key_pressed;

        GP142_open();             /* Open and initialize the GP142 Graphics Window */
        GP142_logging(LOG_OFF);   /* logging is useful to students during debugging, but annoying during the demo; turn it off */
        drawFractal();            /* draw a cool fractal on the screen */

        while (GP142_await_event(&mouse_x, &mouse_y, &key_pressed)!= GP142_QUIT)
        {}

        GP142_close();            /* Clean up and close graphics window */
        return 0;
    }
    '''

    # Write the SVG content to the file
    with open(fractal, 'w') as svg_file:  # Opens a file for writing. Creates the file if it does not exist. Overwrites the file if it exists.
        svg_file.write(svg_content)

    # Print the SVG content to the console
    #print(svg_content)

# Example usage
create_svg("fractal.svg")   # creates an SVG file called example.svg

In [ ]:
# Writing the above code to matach all the other code
def affine_transformation(matrix, vector):
    transformed_vector = [0, 0]
    for i in range(2):
        transformed_vector[i] = matrix[i][0] * vector[0] + matrix[i][1] * vector[1]
    return transformed_vector

def create_svg(fractal):
    # Define the affine transformation matrix and translation vector
    affine_matrix = [[1, 0], [0, 1]]  # Identity matrix for this example
    translation_vector = [50, 50]     # Example translation vector

    # Example input vector
    input_vector = [10, 10]

    # Apply the affine transformation
    transformed_vector = affine_transformation(affine_matrix, input_vector)
    transformed_vector = [transformed_vector[i] + translation_vector[i] for i in range(2)]

    # Write the transformed coordinates to the SVG file
    svg_content = f'''
    <svg width="100" height="100">
        <circle cx="{transformed_vector[0]}" cy="{transformed_vector[1]}" r="5" fill="red" />
    </svg>
    '''

    # Write the SVG content to the file
    with open(fractal, 'w') as svg_file:
        svg_file.write(svg_content)

# Example usage
create_svg("fractal.svg")

In [ ]:
# Example with Affine Transformation

Notes:
- don’t worry about anything but the call to drawFractal() --> going to do all the work
- make a prototype for this function
- it doesn’t need to take any parameters or return a value
- add the functions you defined in Part I to your main.c file (code above)  
- define the drawFractal function by placing some temporary code in the body
- this temporary code should declare some affine transformations and vectors representing points on the graphics screen

Example:
- Step 1: try defining four vectors in a square and connecting them with line segments --> (Draw a line by calling GP142_lineXY. If your background is black, use the color WHITE)
- Step 2: then apply an affine transformation to each vector and connect the resulting four new vectors with line segments
- repeat this with various different transformations and see if you can achieve translation and scaling

In [ ]:
# Step 1 - create a square
def drawFractal(square):
    # Define SVG content as a string
    svg_content = '''
    <svg xmlns="http://www.w3.org/2000/svg" width="100" height="100">
        <line x1="10" y1="10" x2="90" y2="10" stroke="black" stroke-width="2"/>
        <line x1="90" y1="10" x2="90" y2="90" stroke="black" stroke-width="2"/>
        <line x1="90" y1="90" x2="10" y2="90" stroke="black" stroke-width="2"/>
        <line x1="10" y1="90" x2="10" y2="10" stroke="black" stroke-width="2"/>
    </svg>
    '''

    # Write the SVG content to the file
    with open(square, 'w') as svg_file:  # Opens a file for writing. Creates the file if it does not exist. Overwrites the file if it exists.
        svg_file.write(svg_content)

# Example usage
create_svg("square.svg")   # creates an SVG file called example.svg


# Step 2 - apply an affine transformation
import numpy as np

def apply_affine_transformation(point, matrix):
    # Add homogeneous coordinate
    homogeneous_point = np.append(point, [1])
    # Apply transformation
    transformed_point = np.dot(matrix, homogeneous_point)
    # Remove homogeneous coordinate
    return transformed_point[:2]

def create_svg_file(filename):
    # Define the transformation matrix (for example, scaling by a factor of 2)
    transformation_matrix = np.array([[2, 0, 0], [0, 2, 0], [0, 0, 1]])

    # Define the original points of the square
    points = np.array([[10, 10], [90, 10], [90, 90], [10, 90]])

    # Apply the affine transformation to each point
    transformed_points = [apply_affine_transformation(point, transformation_matrix) for point in points]

    # Define SVG content as a string
    svg_content = f'''
    <svg xmlns="http://www.w3.org/2000/svg" width="200" height="200">
        <line x1="{transformed_points[0][0]}" y1="{transformed_points[0][1]}" x2="{transformed_points[1][0]}" y2="{transformed_points[1][1]}" stroke="black" stroke-width="2"/>
        <line x1="{transformed_points[1][0]}" y1="{transformed_points[1][1]}" x2="{transformed_points[2][0]}" y2="{transformed_points[2][1]}" stroke="black" stroke-width="2"/>
        <line x1="{transformed_points[2][0]}" y1="{transformed_points[2][1]}" x2="{transformed_points[3][0]}" y2="{transformed_points[3][1]}" stroke="black" stroke-width="2"/>
        <line x1="{transformed_points[3][0]}" y1="{transformed_points[3][1]}" x2="{transformed_points[0][0]}" y2="{transformed_points[0][1]}" stroke="black" stroke-width="2"/>
    </svg>
    '''

    # Write the SVG content to the file
    with open(filename, 'w') as svg_file:
        svg_file.write(svg_content)

# Example usage
create_svg_file("square_transformed.svg")

In [ ]:
# Reading in an array of Transformations

Notes:
- the user will specify a set of transformations to be used to generate a fractal in a data file “data.txt” that contains the numerical entries for each matrix and vector of each affine transformation
- up to 10 transformations may be specified, one per line of the file, with the exception of the very first line, which will contain a single integer specifying how many transformations to use
- each line after that will contain six double numbers separated by spaces
- the first four numbers on a line will correspond to the entries a, b, c, and d in a transformation’s matrix, and the last two will be the entries in the translation vector
- your program will read in the transformations and store them in a ten-entry array of AffineTransformations


Process:
- delete the temporary code you wrote in drawFractal's body
- in its place, declare an array of AffineTransformations with MAX_TRANSFORMS entries, where MAX_TRANSFORMS is #defined to be 10
- declare an integer that will be the number of transformations actually used
- now define a function getTransforms that takes the arrary of transformations as a parameter and fills this array with transformations read in from data.txt
- this function should return an integer that is the number of transformations used, as read from the first line of the file
- call  getTransforms from drawFractal
- remember to assign the return value to the variable you declared to store the number of transformations used

In [ ]:
# Transformations

def create_svg(transformations):
    svg_content = '''
    #define MAX_TRANSFORMATIONS 10   // specifies the max number of transformations
    typedef struct {
        double matrix[3][3];
        Vector vector;
    } AffineTransformation;   // defined from before

    // this function takes an array of AffineTransformations and a filename as parameters
    int getTransforms(AffineTransformation transforms[], const char* filename) {
        FILE *file = fopen(filename, "r");
        if (file == NULL) {
            printf("Error opening file.\n");
            return 0;
        }

        int num_transforms;
        fscanf(file, "%d", &num_transforms);

        for (int i = 0; i < num_transforms; i++) {
            for (int j = 0; j < 3; j++) {
                for (int k = 0; k < 3; k++) {
                    fscanf(file, "%lf", &transforms[i].matrix[j][k]);
                }
            }
            fscanf(file, "%lf %lf %lf", &transforms[i].vector.x, &transforms[i].vector.y, &transforms[i].vector.z);
        }

        fclose(file);
        return num_transforms;
    }

    void drawFractal() {
        // declare an array of AffineTransformation structs with size MAX_TRANSFORMATIONS to store the transformations
        AffineTransformation transforms[MAX_TRANSFORMS];
        // call getTransforms from drawFractal to read the transformations from the file data.txt and store them in the transforms array
        int num_transforms = getTransforms(transforms, "data.txt");   // use the num_transforms variable to store the number of transforms from the file

        // Drawing logic using the transformations...
    }

    int main() {        // calls drawFractal to start the drawing process
        drawFractal();
        return 0;
    }
    '''

    # Write the SVG content to the file
    with open(transformations, 'w') as svg_file:  # Opens a file for writing. Creates the file if it does not exist. Overwrites the file if it exists.
        svg_file.write(svg_content)

# Example usage
create_svg("transformations.svg")   # creates an SVG file called example.svg

In [ ]:
# Calculating Maximum Recursion Depth

Notes:
- make sure that calculating your fractal image doesn’t take too much time
- don't plot any more points than you have to in order to generate a nice looking image
- You should #define MAX_POINTS, the maximum number of points to plot
- a good number to use for MAX_POINTS is 65536
- you can calculate the maximum depth to which you want your recursive function to be called from this number and the number of transformations used, because each transformation will generate a recursive call for each vector
- this maximum recursion depth turns out to be the logarithm, base b, of MAX_POINTS, where b is the number of transforms
- to calculate log, base b, calculate log and divide by log b
- then cast the result to an int and store this value in an int variable, called max_depth, in drawFractal
- this is what you should have:

      /* calculate the maximun recursion depth: */

      max_depth = ( int )( log( MAX_POINTS ) / log( transforms ) );

- here the variable transforms is the number of transforms being used, as returned by the getTransforms function

In [ ]:
# Generating the Fractal

Step 1:
- Now create a function plotVector that takes a Vector pointer as a parameter and plots the pixel on the screen corresponding to this vector
- You can do this with a single call to GP142_pixelXY
- You will need to cast the Vector’s entries to ints

Step 2:
- Now define a Vector startPoint in drawFractal whose entries are both zero
- Such a vector is known as the zero vector
- This will be the starting point from which all points in the image are generated via a sequences of affine transformations (Note that this point itself is not necessarily part of the image)

Step 3:
- Finally, define the function recursivePlot that takes as parameters a Vector pointer, an array of AffineTransforms, an integer specifying how many transforms there are, an integer specifying current recursion depth, and an integer specifying maximum recursion depth
- The base case for this recursive function should be that the current recursion depth equals the maximum recursion depth
- If this is the case then recursivePlot should simply plot the Vector with a call to plotVector and return

Step 4:
- Otherwise recursivePlot should loop through all the transformations in the array, the number of which was specified in the third parameter
- For each transformation, it should calculate a new vector with a call to AffineTransform and recursively call itself, passing the value of this new vector
- Remember to pass the current recursion depth plus one or the recursion will never reach the base case
- Besides the new vector and recursion depth, all other arguments to the recursive call should be the same

Step 5:
- Call recursivePlot from drawFractal, passing as parameters the zero vector you declared, the array of transformations you read in, the number of transformations read in, zero for the current depth, and the maximum depth that you calculated
- Your program should now be complete, and you can start defining some data files to generate fractals

In [ ]:
# Creating Fractals

In [ ]:
# Here is the data.txt file that generates the snowflake fractal:

4
0.4 0.0 0.0 0.4 -160 0
0.4 0.0 0.0 0.4 160 0
0.4 -0.5 0.5 0.4 0 0
0.4 0.5 -0.5 0.4 0 0

# The Sierpinski Gasket data.txt file:

3
0.5 0.0 0.0 0.5 0 100
0.5 0.0 0.0 0.5 100 -100
0.5 0.0 0.0 0.5 -100 -100

# The Koch Curve data.txt file:

4
.3333 0.0 0.0 .3333 -200 0
.3333 0.0 0.0 .3333 200 0
0.16667 -0.288675 0.288675 0.16667 -50 86.6024
0.16667 0.288675 -0.288675 0.16667 50 86.6024

# A square data.txt file:

8
0.3333 0.0 0.0 0.3333 0 150
0.3333 0.0 0.0 0.3333 0 -150
0.3333 0.0 0.0 0.3333 150 0
0.3333 0.0 0.0 0.3333 -150 0
0.3333 0.0 0.0 0.3333 150 -150
0.3333 0.0 0.0 0.3333 -150 150
0.3333 0.0 0.0 0.3333 150 150
0.3333 0.0 0.0 0.3333 -150 -150

# A pentagon data.txt file:

5
0.38 0.0 0.0 0.38 100 0
0.38 0.0 0.0 0.38 30.9017 95.1057
0.38 0.0 0.0 0.38 -80.9017 58.7785
0.38 0.0 0.0 0.38 -80.9017 -58.7785
0.38 0.0 0.0 0.38 30.9017 -95.1057

# Dragon Fire data.txt file:

3
0.5 0.0 0.0 0.8 0.0 50
0.5 0.2 -0.2 0.5 -100 -100
0.5 -0.2 0.2 0.5 100 -100

SyntaxError: invalid syntax (<ipython-input-9-089ae40c2acd>, line 4)

In [ ]:
# How to control the fractals you generate

Notes:
- The secret is to consider the layout of the self-similar pieces of the fractal
- First of all, how many such pieces are there?
- The number of pieces tells you how many transformations you will need
- For example, the Sierpinski Gasket consists of three pieces identical to itself, each ½ scale with one on top of the other two
- Thus you need three transformations, and each transformation needs to scale by 0.5, in both the x and y dimensions
- The matrix part of an affine transformation is responsible for scaling
- The key to creating a matrix that will scale by a certain factor is to take a matrix that scales by a factor of one and then multiply each entry by the desired factor
- The matrix that scales by a factor of one is called the identity matrix
      [1 0]
      [0 1]
- Thus the matrix that scales by a factor of 0.5 is just 0.5 times this matrix:
      [0.5  0 ]
      [ 0  0.5]
- Note that it is also possible to scale in the x-dimension by a different factor than in the y-dimension. The top row of the matrix effects x-coordinates, and the bottom row affects y-coordinates
- Next you need to think about how the three pieces are positioned
- This is where the translation comes into the picture
- The first transformation of the Gasket has the translation vector (0, 100)
- This tells you that the translation is 100 units straight up
- The second is (100, -100), corresponding to down and to the right
- The third is (-100, -100), indicating a translation down and to the left
- Thus the image will have a triangular arrangement
- Generating the Gasket doesn’t involve any rotation or skew
- However, generating the Koch Curve does. To do a rotation, use a matrix like this:
      [cos(theta)  -sin(theta)]
      [sin(theta)   cos(theta)]
- You need to calculate actual numbers to put in the data.txt file
- The resulting rotation matrix needs to be adjusted for scaling, by multiplying each entry by the desired scale factor
- The Koch fractal uses four transformations
- The first two translate left and right and scale by 1/3, but do not rotate
- The second two translate up, with one also translating left and rotating by pi/3 radians, and the other translating right and rotating by –pi/3 radians
- These two scale by 1/3 also
- The affect of doing such transformations on a horizontal line of length 3 would be the generation of the same line but with the middle one third removed and replaced by an equilateral triangle of side length 1, and no base
- When we’re drawing the Koch fractal, we recursively apply these transformations over and over, and thus see the shape that would be generated by repeating this process for each new line segment ad infinitum